In [68]:
from sentence_transformers import SentenceTransformer
sentences = ["Mikrotik bot fue creado para solucionar problemas", "Fue creado por Juan y Luis"]

encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = encoder.encode(sentences)
print(embeddings)


[[-6.51995614e-02  6.05133586e-02  1.79699366e-03 -2.11219750e-02
  -5.87918423e-02 -1.22480020e-02  4.41434160e-02  1.38162985e-01
  -6.35557994e-02 -2.84093935e-02  2.94391364e-02 -1.86998621e-02
   7.93902576e-02 -3.44210714e-02 -1.73610020e-02  2.74901800e-02
  -6.72735497e-02  1.88886560e-02 -2.12969221e-02 -4.46643792e-02
   5.74383475e-02 -1.13737829e-01 -7.40510300e-02 -1.76676847e-02
  -6.63157925e-02  6.56991899e-02  5.84233329e-02  1.03078131e-02
   9.50153265e-03  1.02672260e-03  5.41067235e-02  6.32527843e-02
  -1.25984810e-02  3.45091745e-02  4.84431125e-02  4.40974385e-02
  -1.00096479e-01 -2.05638018e-02  7.98964966e-03  5.61853535e-02
  -1.08684629e-01 -6.23195842e-02 -1.35199353e-02 -2.90265139e-02
   9.97233540e-02 -2.92712357e-03 -1.20487353e-02  9.22772065e-02
   6.03910070e-03 -5.81868477e-02 -1.06771603e-01 -9.55989677e-03
  -4.75460663e-02  2.72578076e-02  2.75665130e-02 -8.85460973e-02
   5.76454122e-03  4.43496518e-02  6.09696545e-02 -1.00269884e-01
   1.26418

In [69]:
embeddings_length = len(embeddings[0])
print(embeddings_length)

384


In [70]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client.http.models import Distance, VectorParams

load_dotenv()

qdrant_api_endpoint = os.getenv("QDRANT_API_ENDPOINT")
qdrant_api_key = os.getenv("QDRANT_API_KEY")

client = QdrantClient(
    url=qdrant_api_endpoint,
    api_key=qdrant_api_key
)
collection_name = "mikrotik-bot-info"
client.delete_collection(collection_name=collection_name)
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embeddings_length,distance=Distance.COSINE)
)

True

In [71]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [72]:
from qdrant_client import models, QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings
import uuid

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

sentences = ["Mikrotik bot fue creado para solucionar problemas", "Fue creado por Juan y Luis"]


vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)

In [84]:
# read mikrotik_bot.txt
with open("mikrotik_bot.txt", "r",encoding="latin-1") as file:
    mikrotik_bot_text = file.read()

In [85]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 150, chunk_overlap=10, separator='.', strip_whitespace=False)

text_documents = text_splitter.create_documents([mikrotik_bot_text])

In [94]:
text_splitter

In [87]:
len(text_documents)

[Document(metadata={}, page_content='MIKROTIK BOT\nNombre: Mikrotik Bot.\nMotivo de creaciÃ³n: \n-\tDelegar preguntas repetitivas.\n-\tReducciÃ³n de carga en agentes humanos'),
 Document(metadata={}, page_content='\n-\tMejora en la eficiencia.\n-\tConsistencia en respuestas gracias a la bÃºsqueda en base de datos actualizada.\nLugar de creaciÃ³n: Lima, PerÃº'),
 Document(metadata={}, page_content='\nCreadores: Juan Rodriguez y Luis Huaman.\nProfesor: Espichan Vilca James Muller.\nTrabajo Final del curso de creaciÃ³n de aplicaciones con IA para IDAT'),
 Document(metadata={}, page_content='\nArquitectura del bot:\nDe manera general el chatbot tiene: router, main agent, compare agent, about agent'),
 Document(metadata={}, page_content='\nEl bot estÃ¡ constituido por un router (inicialmente estaba un rephraser que nos ayudaba a mejorar la pregunta, sin embargo, debido a diferentes errores se ha quitado el rephrser)'),
 Document(metadata={}, page_content='\nEste router contiene 3 posibles r

In [88]:
vector_store.add_documents(
    documents=text_documents,
    ids=[str(uuid.uuid4()) for _ in range(len(text_documents))])

['0a98507b-ede4-4b62-bcb0-968e8c0dad19',
 'd8858544-0fd7-4de9-a40b-8c5141f96d37',
 '44f41870-88a9-45be-b8c0-889ee44c007e',
 '8dba1736-0bf3-4abb-93c1-72a50421307f',
 '8ec5958c-5a96-40d6-a584-f04c63eb3d91',
 '65bf4564-d4af-4021-80f3-3cabf05a71fe',
 '8b3ee430-0aeb-4bbd-9059-999bf991ad63',
 '0dca9998-a14e-4448-bb95-9ac403c26bfb',
 '58e52071-9dd2-43a7-b9f4-e7d221e2d7f4',
 'f3f50a85-e613-4573-87d6-077dcf8da569',
 '6fbca2c6-0716-47b7-81a9-bd16ebc61310',
 'e0a54a06-68fc-4d5e-b644-3e79f1368a46']

In [95]:
vector_store.similarity_search(query="Como llegas a comparar las cosas?", k=4)

[Document(metadata={'_id': 'e0a54a06-68fc-4d5e-b644-3e79f1368a46', '_collection_name': 'mikrotik-bot-info'}, page_content='\nCompare agent: este agente compara los productos que el usuario ha agregado a la conversaciÃ³n como contexto.\n\n\n\n\n\n'),
 Document(metadata={'_id': '8ec5958c-5a96-40d6-a584-f04c63eb3d91', '_collection_name': 'mikrotik-bot-info'}, page_content='\nEl bot estÃ¡ constituido por un router (inicialmente estaba un rephraser que nos ayudaba a mejorar la pregunta, sin embargo, debido a diferentes errores se ha quitado el rephrser)'),
 Document(metadata={'_id': 'd8858544-0fd7-4de9-a40b-8c5141f96d37', '_collection_name': 'mikrotik-bot-info'}, page_content='\n-\tMejora en la eficiencia.\n-\tConsistencia en respuestas gracias a la bÃºsqueda en base de datos actualizada.\nLugar de creaciÃ³n: Lima, PerÃº'),
 Document(metadata={'_id': '44f41870-88a9-45be-b8c0-889ee44c007e', '_collection_name': 'mikrotik-bot-info'}, page_content='\nCreadores: Juan Rodriguez y Luis Huaman.\nPr